In [ ]:
from fastai.vision.all import *

In [ ]:
df_full = pd.read_csv('../input/bms-molecular-translation/train_labels.csv')[:10_000]

In [ ]:
df_full

In [ ]:
! pip install chemparse

In [ ]:
import chemparse

df = df_full.join(
    pd.json_normalize(df_full['InChI'].str.split('/',expand=True)[1].apply(chemparse.parse_formula)))
df.drop('InChI',axis=1,inplace=True)
df.head(10)

In [ ]:
df.fillna(0,inplace=True)
df

In [ ]:
df[df.N>=1]

In [ ]:
df

In [ ]:
dblock = DataBlock(
    blocks = (ImageBlock,RegressionBlock),
    get_x = lambda x : f'../input/drawing-molecules-with-rdkit-inchi-to-png/data/{x["image_id"]}.png',
    get_y = lambda x : x["N"]
)

In [ ]:
dls = dblock.dataloaders(df)

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls,resnet18)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3,lr_max =3e-2)

In [ ]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)   
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
def gradcam(path):
    img = PILImage.create(path)
    x, = first(dls.test_dl([img]))

    with HookBwd(learn.model[0]) as hookg:
        with Hook(learn.model[0]) as hook:
            output = learn.model.eval()(x.cuda())
            act = hook.stored
        output.backward()
        grad = hookg.stored

    w = grad[0].mean(dim=[1,2], keepdim=True)
    cam_map = (w * act[0]).sum(0)

    x_dec = TensorImage(dls.train.decode((x,))[0][0])
    _,ax = plt.subplots()
    x_dec.show(ctx=ax)
    ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,512,512,0),
                  interpolation='bilinear', cmap='magma');

In [ ]:
files = get_image_files('../input/drawing-molecules-with-rdkit-inchi-to-png')
gradcam(files[222])

In [ ]:
gradcam(files[3321])